In [1]:
%matplotlib widget

In [151]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

In [124]:
folder = 'data'
file_names = ['가평_2019.xlsx', '의암호_2019.xlsx']

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

for i in range(len(file_names)):
    path = os.path.join(folder, file_names[i])

    df_full.append(pd.read_excel(path))
    df.append(df_full[i].iloc[:, 2:11])
    date_time = pd.to_datetime(df_full[i].iloc[:, 0], format='%Y.%m.%d %H:%M')
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[i]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[i]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[i]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[i]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [126]:
df[1]


,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,3.0,7.2,91.0,12.2,0.7,1.3,1.469,0.001,8.0,-7.071068e-01,-7.071068e-01,-0.004430,0.999990
1,3.0,7.2,91.0,12.1,0.8,1.3,1.526,0.002,8.1,-8.660254e-01,-5.000000e-01,-0.003713,0.999993
2,3.0,7.2,91.0,12.1,0.8,1.3,1.623,0.002,8.2,-9.659258e-01,-2.588190e-01,-0.002996,0.999996
3,3.1,7.2,90.0,12.1,0.8,1.3,1.637,0.003,8.4,-1.000000e+00,-2.466750e-12,-0.002279,0.999997
4,3.1,7.2,90.0,12.1,0.8,1.3,1.563,0.002,8.5,-9.659258e-01,2.588190e-01,-0.001563,0.999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,4.8,7.7,99.0,12.3,0.7,1.5,1.430,0.003,NaN,5.000000e-01,-8.660254e-01,-0.012185,0.999926
8756,4.8,7.7,100.0,12.4,0.7,1.5,1.421,0.003,NaN,2.588190e-01,-9.659258e-01,-0.011468,0.999934
8757,4.7,7.7,100.0,12.4,0.7,1.5,1.457,0.002,NaN,2.543654e-12,-1.000000e+00,-0.010752,0.999942
8758,4.7,7.7,101.0,12.4,0.7,1.5,1.447,0.003,NaN,-2.588190e-01,-9.659258e-01,-0.010035,0.999950


In [127]:
# normalize data

df_all = pd.concat((df1, df2))
df_all

train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std

In [165]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.496853,-0.945645,-1.065617,0.998777,-0.669890,-0.875013,0.872656,-0.515611,-1.026082,-9.999715e-01,-9.999715e-01,-0.006269,1.415570
1,-1.496853,-0.945645,-1.065617,0.998777,-0.591824,-0.875013,1.020773,-0.249286,-0.988738,-1.224710e+00,-7.070866e-01,-0.005255,1.415574
2,-1.496853,-0.945645,-1.065617,0.998777,-0.747956,-0.875013,0.850980,-0.781935,-1.026082,-1.365986e+00,-3.660150e-01,-0.004242,1.415577
3,-1.510410,-0.945645,-1.065617,0.998777,-0.669890,-0.875013,0.887106,-0.249286,-1.013634,-1.414173e+00,-3.519073e-12,-0.003229,1.415580
4,-1.496853,-0.945645,-1.065617,0.947057,-0.669890,-0.875013,0.883494,-0.515611,-1.013634,-1.365986e+00,3.660150e-01,-0.002215,1.415582
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,-1.388397,-0.289409,0.698099,0.998777,-0.201494,0.704985,0.681187,-1.048260,1.326626,7.070866e-01,-1.224710e+00,-0.017232,1.415479
8756,-1.401954,-0.289409,0.698099,0.998777,-0.201494,0.309986,0.710088,-1.314584,1.227040,3.660150e-01,-1.365986e+00,-0.016219,1.415491
8757,-1.415511,-0.289409,0.698099,0.998777,-0.201494,0.309986,0.753440,-1.048260,1.152351,3.576374e-12,-1.414173e+00,-0.015206,1.415502
8758,-1.415511,-0.289409,0.698099,0.998777,-0.201494,0.309986,0.699250,-1.048260,1.177248,-3.660150e-01,-1.365986e+00,-0.014193,1.415512


In [139]:
input_slice = slice(0,3)
input_indices = np.arange(10)[input_slice]
input_indices

array([0, 1, 2])

In [142]:
train_df = df[0]
val_df = df[1]
test_df = df[1]

In [143]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [144]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [154]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [146]:
w2 = WindowGenerator(input_width=6, label_width=1, shift=1,
                     label_columns=None)
w2

Total window size: 7
Input indices: [0 1 2 3 4 5]
Label indices: [6]
Label column name(s): None

In [147]:
# Stack three slices, the length of the total window:
example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])


example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'labels shape: {example_labels.shape}')

All shapes are: (batch, time, features)
Window shape: (3, 7, 13)
Inputs shape: (3, 6, 13)
labels shape: (3, 1, 13)


In [148]:
w2.example = example_inputs, example_labels

In [198]:
w2.plot(plot_col='수온')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [201]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [202]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [236]:
from tensorflow import keras

class GainDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=0,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 alpha=100.):
        'Initialization'
        window_size = input_width

        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany == 1] = np.nan
            cums.append(cum)
            
        self.data = np.concatenate(data_list)
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        self.data_m = binary_sampler(miss_rate, self.data.shape)
        self.batch_size = batch_size
        #print(len(data_idx))
        self.batch_idx = sample_batch_index(self.no, self.batch_size)
        self.batch_id = 0
        #print(data_idx)
        #print(self.data[data_idx[0]])
            
            
            
        # Define mask matrix
        #self.data_m = 1-np.isnan(data_x)

        # Normalization
        #self.norm_data, self.norm_parameters = normalization(data_x)
        #self.norm_data_x = np.nan_to_num(self.norm_data, 0)
        #norm_data_x = norm_data_x.astype(np.float32)
        #data_x = data_x.astype(np.float32)

        #self.idx = 0
        #self.batch_cnt = self.no // self.batch_size
        #self.shuffle_idx = np.random.permutation(self.no)

    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        '''
        batch_idx = self.shuffle_idx[self.idx*self.batch_size:(self.idx+1)*self.batch_size]
        self.idx = (self.idx+1)%self.batch_cnt
        if (self.idx == 0):
            self.shuffle_idx = np.random.permutation(self.no)
        '''
        i = self.batch_idx[self.batch_id]
        self.batch_id += 1
        self.batch_id %= self.batch_size
        idx1 = self.data_idx[i]
        idx2 = self.data_idx[i]+self.input_width
        #print(idx1, idx2)
        
        Y_mb = self.data[idx1:idx2]
        M_mb = self.data_m[idx1:idx2]
        X_mb = Y_mb.copy()
        X_mb[M_mb == 1] = np.nan
        #print(X_mb.shape, Y_mb.shape)
        
        return [X_mb, Y_mb]
        #X_mb = tf.convert_to_tensor(X_mb, dtype=tf.float32)
        #M_mb = tf.convert_to_tensor(M_mb, dtype=tf.float32)

        # Sample random vectors  
        #Z_mb = uniform_sampler(0, 0.01, self.batch_size, self.dim)
        #Z_mb = tf.keras.backend.random_uniform(shape=(self.batch_size,self.dim), minval=0.0, maxval=0.01)
        # Sample hint vectors
        #H_mb_temp = binary_sampler(self.hint_rate, self.batch_size, self.dim)
        #print('H_mb_temp.shape=',H_mb_temp.shape)
        #print('M_mb.shape=', M_mb.shape)

        #H_mb = M_mb * H_mb_temp
        
        # Combine random vectors with observed vectors
        #X_mb = M_mb * X_mb + (1-M_mb) * Z_mb

        #return X_mb, M_mb, H_mb
        #return {
          #'model_input_x': X_mb,
          #'model_input_m': M_mb,
          #'model_input_h': H_mb,
        #}

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

dgen = GainDataGenerator(df)

In [237]:
it = iter(dgen)

In [238]:
x,y = next(it)

In [240]:
y

array([[-1.42906778e+00, -7.06640655e-02,  1.09003555e+00,
         1.98147070e+00, -7.47956201e-01, -2.06001108e+00,
         1.16889041e+00,  1.08233692e+00, -9.51393044e-01,
        -5.79536207e-12,  1.41417320e+00,  1.08901592e+00,
         9.02976152e-01],
       [-1.38839687e+00, -7.06640655e-02,  1.18801977e+00,
         2.08491211e+00, -7.47956201e-01, -1.66501163e+00,
         1.25559317e+00,  5.49687735e-01, -1.02608219e+00,
         3.66014958e-01,  1.36598642e+00,  1.08966177e+00,
         9.02194809e-01],
       [-1.37483990e+00, -7.06640655e-02,  1.09003555e+00,
         2.08491211e+00, -7.47956201e-01, -2.06001108e+00,
         1.32062023e+00,  8.16012326e-01, -1.12566772e+00,
         7.07086601e-01,  1.22470992e+00,  1.09030707e+00,
         9.01413003e-01],
       [-1.37483990e+00, -7.06640655e-02,  1.09003555e+00,
         2.08491211e+00, -7.47956201e-01, -2.06001108e+00,
         1.35674638e+00,  2.83363144e-01, -1.15056410e+00,
         9.99971461e-01,  9.99971461e

In [ ]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [281]:

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense

class Gain(keras.Model):
    def __init__(self, shape, alpha=100., load=False, **kwargs):
        super(Gain, self).__init__(**kwargs)
        self.dim = sum(shape)
        self.h_dim = self.dim
        self.build_generator()
        self.build_discriminator()

    ## GAIN models
    def build_generator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        x = Input(shape=(self.dim,), name='generator_input_x')
        m = Input(shape=(self.dim,), name='generator_input_m')

        a = Concatenate()([x, m])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        G_prob = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        x = Input(shape=(self.dim,), name='discriminator_input_x')
        h = Input(shape=(self.dim,), name='discriminator_input_h')

        a = Concatenate()([x, h])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        D_prob = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        shape = inputs.shape
        dims = sum(shape[1:])
        input_width = shape[1]
        print(inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        x = keras.layers.Reshape((dims,))(inputs)
        
        m = keras.backend.cast(tf.math.is_nan(x), dtype=tf.float32)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        imputed_data = m*x + (1-m)*x
        
        return imputed_data
    
    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 28 * 28
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [282]:
x = np.random.random((1,10))
model = Gain(shape=(10,))
model.compile()
model.predict(x)

(None, 10)


array([[0.6599658 , 0.8402047 , 0.7367559 , 0.84127563, 0.8828531 ,
        0.19044958, 0.8602449 , 0.12705377, 0.08603664, 0.5487207 ]],
      dtype=float32)

In [83]:
df1_full.iloc[:, 0:1]

,측정날짜
0,2019.01.01 00:00
1,2019.01.01 01:00
2,2019.01.01 02:00
3,2019.01.01 03:00
4,2019.01.01 04:00
...,...
8755,2019.12.31 19:00
8756,2019.12.31 20:00
8757,2019.12.31 21:00
8758,2019.12.31 22:00


In [66]:
df2.isna().astype(int).sum()

수온         263
수소이온농도     263
전기전도도      263
용존산소       858
탁도         297
총유기탄소     2785
총질소        619
총인         621
클로로필-a     972
dtype: int64

In [99]:
date_time1 = pd.to_datetime(df1_full.iloc[:, 0], format='%Y.%m.%d %H:%M')
date_time2 = pd.to_datetime(df2_full.iloc[:, 0], format='%Y.%m.%d %H:%M')

In [100]:
timestamp_s1 = date_time1.map(datetime.datetime.timestamp)
timestamp_s2 = date_time2.map(datetime.datetime.timestamp)

In [101]:
day = 24*60*60
year = (365.2425)*day

df1['Day sin'] = np.sin(timestamp_s1 * (2 * np.pi / day))
df1['Day cos'] = np.cos(timestamp_s1 * (2 * np.pi / day))
df1['Year sin'] = np.sin(timestamp_s1 * (2 * np.pi / year))
df1['Year cos'] = np.cos(timestamp_s1 * (2 * np.pi / year))

df2['Day sin'] = np.sin(timestamp_s2 * (2 * np.pi / day))
df2['Day cos'] = np.cos(timestamp_s2 * (2 * np.pi / day))
df2['Year sin'] = np.sin(timestamp_s2 * (2 * np.pi / year))
df2['Year cos'] = np.cos(timestamp_s2 * (2 * np.pi / year))

In [102]:
df2

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,3.0,7.2,91.0,12.2,0.7,1.3,1.469,0.001,8.0,-7.071068e-01,-7.071068e-01,-0.004430,0.999990
1,3.0,7.2,91.0,12.1,0.8,1.3,1.526,0.002,8.1,-8.660254e-01,-5.000000e-01,-0.003713,0.999993
2,3.0,7.2,91.0,12.1,0.8,1.3,1.623,0.002,8.2,-9.659258e-01,-2.588190e-01,-0.002996,0.999996
3,3.1,7.2,90.0,12.1,0.8,1.3,1.637,0.003,8.4,-1.000000e+00,-2.466750e-12,-0.002279,0.999997
4,3.1,7.2,90.0,12.1,0.8,1.3,1.563,0.002,8.5,-9.659258e-01,2.588190e-01,-0.001563,0.999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,4.8,7.7,99.0,12.3,0.7,1.5,1.430,0.003,NaN,5.000000e-01,-8.660254e-01,-0.012185,0.999926
8756,4.8,7.7,100.0,12.4,0.7,1.5,1.421,0.003,NaN,2.588190e-01,-9.659258e-01,-0.011468,0.999934
8757,4.7,7.7,100.0,12.4,0.7,1.5,1.457,0.002,NaN,2.543654e-12,-1.000000e+00,-0.010752,0.999942
8758,4.7,7.7,101.0,12.4,0.7,1.5,1.447,0.003,NaN,-2.588190e-01,-9.659258e-01,-0.010035,0.999950
